# 🧱 Oracle SQL*Plus Setup on Amazon Linux 2023 using Docker

This notebook provides the minimal and working EC2 **User Data Script** to automatically deploy **Oracle XE (SQL*Plus)** inside a Docker container on **Amazon Linux 2023**.

---

## ✅ EC2 User Data Script

Copy the following script into the **User Data** section when launching your EC2 instance.

```bash
#!/bin/bash -xe
# ===== Oracle SQL*Plus on Amazon Linux 2023 (Fixed Version) =====

# Update and install Docker
dnf update -y
dnf install -y docker

# Enable and start Docker
systemctl enable docker
systemctl start docker

# Add ec2-user to Docker group
usermod -aG docker ec2-user

# Create and fix permissions for persistent Oracle data
mkdir -p /home/ec2-user/oracle-data
chmod 777 /home/ec2-user/oracle-data

# Pull Oracle XE image (includes SQL*Plus)
docker pull gvenzl/oracle-xe:21-slim

# Run Oracle XE container
docker run -d \
  --name oracle-xe \
  -p 1521:1521 -p 5500:5500 \
  -e ORACLE_PASSWORD=oracle \
  -v /home/ec2-user/oracle-data:/opt/oracle/oradata \
  --shm-size=1g \
  gvenzl/oracle-xe:21-slim

echo "Oracle XE container started. Initialization takes around 2-3 minutes."
echo "Check logs using: docker logs -f oracle-xe"
```

---

## 🧾 Verification Steps

After the instance starts, connect via SSH and verify:

```bash
sudo su ec2-user
docker ps -a
docker logs -f oracle-xe     # Wait until 'DATABASE IS READY TO USE!'
docker exec -it oracle-xe sqlplus system/oracle@//localhost:1521/XEPDB1
```

Run a simple test query in SQL*Plus:
```sql
SELECT sysdate FROM dual;
```

---

## ⚙️ Requirements

| Item | Recommendation |
|------|----------------|
| Instance Type | t3.small (2 GB RAM) or higher |
| Storage | 20 GB minimum |
| OS | Amazon Linux 2023 |
| Docker Image | gvenzl/oracle-xe:21-slim |

---

## ✅ Summary

- Installs Docker on Amazon Linux 2023
- Sets correct directory permissions for Oracle data
- Deploys Oracle XE container with SQL*Plus ready to use
- Ideal for quick training or testing environments

---